In [3]:
# Monthly Bill Generator Notebook

from datetime import datetime
from calendar import monthrange
from collections import defaultdict

def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d").date()

def month_date_range(target_month):
    year, month = map(int, target_month.split('-'))
    start = datetime(year, month, 1).date()
    end = datetime(year, month, monthrange(year, month)[1]).date()
    return start, end

def get_overlap_days(start1, end1, start2, end2):
    latest_start = max(start1, start2)
    earliest_end = min(end1, end2)
    delta = (earliest_end - latest_start).days + 1
    return max(0, delta)

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    bill_start, bill_end = month_date_range(target_month)
    grouped = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        try:
            item_start = parse_date(item["start_date"])
            item_end = parse_date(item["stop_date"])
        except:
            continue

        overlap_days = get_overlap_days(item_start, item_end, bill_start, bill_end)
        if overlap_days == 0:
            continue  # not active in this month

        total_days_in_month = (bill_end - bill_start).days + 1
        qty = int(item["qty"])
        rate = float(item["rate"])

        prorated_amount = round((rate * qty * overlap_days) / total_days_in_month, 2)

        billing_period = f"{bill_start} to {bill_end}"
        key = (item["item_code"], rate, billing_period)

        grouped[key]["qty"] += qty
        grouped[key]["amount"] += prorated_amount

    line_items = []
    total_revenue = 0.0
    for (item_code, rate, billing_period), data in grouped.items():
        amount = round(data["amount"], 2)
        total_revenue += amount
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": data["qty"],
            "amount": amount,
            "billing_period": billing_period
        })

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

# Sample test
from pprint import pprint

item_list = [  # (paste the full item_list you were given here)
    {
     "idx": 1,
     "item_code": "Executive Desk (4*2)",
     "sales_description": "Dedicated Executive Desk",
     "qty": 10,
     "rate": "1000",
     "amount": "10000",
     "start_date": "2023-11-01",
     "stop_date": "2024-10-17",
    },
    # ... (include all 16 items here)
]

result = generate_monthly_bill(item_list, "2024-11")
pprint(result)


{'line_items': [], 'total_revenue': 0.0}
